# Hierarchical Clustering

**Hierarchical clustering** refers to a class of clustering methods that seek to build a **hierarchy** of clusters, in which some clusters contain others. In this assignment, we will explore a top-down approach, recursively bipartitioning the data using k-means.

## Import packages

The following code block will check if you have the correct version of GraphLab Create. Any version later than 1.8.5 will do. To upgrade, read [this page](https://turi.com/download/upgrade-graphlab-create.html).

In [1]:
%matplotlib inline

In [2]:
import json
import numpy as np
import pandas as pd

In [3]:
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

## Load the Wikipedia dataset

In [4]:
wiki = pd.read_csv('./data/people_wiki.csv')
wiki.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


As we did in previous assignments, let's extract the TF-IDF features:

In [5]:
def load_sparse_csr(filename):
    
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape )

In [6]:
tf_idf = load_sparse_csr('./data/4_tf_idf.npz')
tf_idf = normalize(tf_idf)

In [7]:
with open('./data/4_map_index_to_word.json') as f:
    idx_word_map_dict = json.load(f)

map_index_to_word = pd.Series(list(idx_word_map_dict.keys()), index=idx_word_map_dict.values())

## Bipartition the Wikipedia dataset using k-means

Recall our workflow for clustering text data with k-means:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with some value of k.
4. Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

Let us modify the workflow to perform bipartitioning:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with k=2.
4. Divide the data matrix into two parts using the cluster assignments.
5. Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
6. Visualize the bipartition of data.

We'd like to be able to repeat Steps 3-6 multiple times to produce a **hierarchy** of clusters such as the following:
```
                      (root)
                         |
            +------------+-------------+
            |                          |
         Cluster                    Cluster
     +------+-----+             +------+-----+
     |            |             |            |
   Cluster     Cluster       Cluster      Cluster
```
Each **parent cluster** is bipartitioned to produce two **child clusters**. At the very top is the **root cluster**, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

* `dataframe`: a subset of the original dataframe that correspond to member rows of the cluster
* `matrix`: same set of rows, stored in sparse matrix format
* `centroid`: the centroid of the cluster (not applicable for the root cluster)

Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [8]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: original dataframe
                * matrix:    same data, in matrix format
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=1)
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
                                                      data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    dataframe_left_child, dataframe_right_child = dataframe.loc[cluster_assignment==0,:], \
                                                  dataframe.loc[cluster_assignment==1,:]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return (cluster_left_child, cluster_right_child)

The following cell performs bipartitioning of the Wikipedia dataset. Allow 20-60 seconds to finish.

Note. For the purpose of the assignment, we set an explicit seed (`seed=1`) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [9]:
wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=6, seed=1)

Let's examine the contents of one of the two clusters, which we call the `left_child`, referring to the tree visualization above.

In [10]:
left_child['dataframe'].head()

,URI,name,text
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...
5,<http://dbpedia.org/resource/Sam_Henderson>,Sam Henderson,sam henderson born october 18 1969 is an ameri...


And here is the content of the other cluster we named `right_child`.

In [11]:
right_child['dataframe'].head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
17,<http://dbpedia.org/resource/Paddy_Dunne_(Gael...,Paddy Dunne (Gaelic footballer),paddy dunne was a gaelic football player from ...
21,<http://dbpedia.org/resource/Ceiron_Thomas>,Ceiron Thomas,ceiron thomas born 23 october 1983 is a welsh ...
22,<http://dbpedia.org/resource/Adel_Sellimi>,Adel Sellimi,adel sellimi arabic was born on 16 november 19...
25,<http://dbpedia.org/resource/Vic_Stasiuk>,Vic Stasiuk,victor john stasiuk born may 23 1929 is a reti...


## Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [12]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: SFrame specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    for i in range(5):
        print('{0:s}:{1:.3f}'.format(map_index_to_word[idx[i]], centroid[idx[i]])),
    print('')
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in range(8):
        text = ' '.join(wiki_subset.iloc[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset.iloc[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

Let's visualize the two child clusters:

In [13]:
display_single_tf_idf_cluster(left_child, map_index_to_word)

she:0.027
her:0.018
music:0.013
film:0.011
university:0.011

* Kayee Griffin                                      0.97400
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* Vanessa Gilmore                                    0.97570
  vanessa diane gilmore born october 1956 is a judge on the united states district court for
   the southern district of texas she was appointed to
* Ellen Christine Christiansen                       0.97746
  ellen christine christiansen born 10 december 1964 is a norwegian politician representing 
  the conservative party and formerly the progress partyborn in oslo she finished her
* Anina (singer)                                     0.97806
  anina is a norwegian singer and songwriter in 2007 anina got her music degree from the pre
  stigious songwriters academy musikmakarna in sweden the same year
* Lady Gaga                            

In [14]:
display_single_tf_idf_cluster(right_child, map_index_to_word)

league:0.043
season:0.038
football:0.031
team:0.030
played:0.028

* Chris Day                                          0.95257
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Ashley Prescott                                    0.95418
  ashley prescott born 11 september 1972 is a former australian rules footballer he played w
  ith the richmond and fremantle football clubs in the afl between
* Eric Fox                                           0.95662
  eric hollis fox born august 15 1963 in lemoore california is an american professional base
  ball coach the 5 ft 10 in 178 m 180 lb
* Alex Lawless                                       0.95715
  alexander graham alex lawless born 26 march 1985 is a welsh professional footballer who pl
  ays for luton town as a midfielderlawless began his career with
* Ted Silva                                          0.95875
  theodor

The left cluster consists of athletes, whereas the right cluster consists of non-athletes. So far, we have a single-level hierarchy consisting of two clusters, as follows:

```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
                 Athletes                                      Non-athletes
```

Is this hierarchy good enough? **When building a hierarchy of clusters, we must keep our particular application in mind.** For instance, we might want to build a **directory** for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of athletes and non-athletes are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both the `athletes` and `non-athletes` clusters.

## Perform recursive bipartitioning

### Cluster of athletes

To help identify the clusters we've built so far, let's give them easy-to-read aliases:

In [15]:
athletes = right_child
non_athletes = left_child

Using the bipartition function, we produce two child clusters of the athlete cluster:

In [16]:
# Bipartition the cluster of athletes
left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=6, seed=1)

The left child cluster mainly consists of baseball players:

In [17]:
display_single_tf_idf_cluster(left_child_athletes, map_index_to_word)

baseball:0.117
league:0.103
major:0.050
games:0.047
sox:0.045

* Eric Fox                                           0.90350
  eric hollis fox born august 15 1963 in lemoore california is an american professional base
  ball coach the 5 ft 10 in 178 m 180 lb
* Tom Flanigan (baseball)                            0.90430
  thomas anthony flanigan born september 6 1934 at cincinnati ohio is a retired american pro
  fessional baseball player a 6 ft 3 in 191 m 175
* Ted Silva                                          0.90783
  theodore a silva born august 4 1974 in inglewood california has held numerous roles in ama
  teur and professional baseball he has played in the minor
* Sean DePaula                                       0.90902
  sean michael depaula born november 7 1973 is an american former major league baseball play
  er a pitcher depaula played for the cleveland indians appearing in
* Robert Person                                      0.90979
  robert alan person is a former major l

On the other hand, the right child cluster is a mix of players in association football, Austrailian rules football and ice hockey:

In [18]:
display_single_tf_idf_cluster(right_child_athletes, map_index_to_word)

football:0.038
season:0.036
team:0.033
league:0.031
played:0.029

* Chris Day                                          0.95198
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Ashley Prescott                                    0.95233
  ashley prescott born 11 september 1972 is a former australian rules footballer he played w
  ith the richmond and fremantle football clubs in the afl between
* Alex Lawless                                       0.95581
  alexander graham alex lawless born 26 march 1985 is a welsh professional footballer who pl
  ays for luton town as a midfielderlawless began his career with
* Gordon Astall                                      0.95953
  gordon astall born 22 september 1927 is an english former professional footballer he playe
  d as an outside right and represented the football league the england
* Gary Kelly (footballer, born 1966)   

Our hierarchy of clusters now looks like this:
```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
                 Athletes                                      Non-athletes
                    +
                    |
        +-----------+--------+
        |                    |
        |            association football/
        +          Austrailian rules football/
     baseball             ice hockey
```

Should we keep subdividing the clusters? If so, which cluster should we subdivide? To answer this question, we again think about our application. Since we organize our directory by topics, it would be nice to have topics that are about as coarse as each other. For instance, if one cluster is about baseball, we expect some other clusters about football, basketball, volleyball, and so forth. That is, **we would like to achieve similar level of granularity for all clusters.**

Notice that the right child cluster is more coarse than the left child cluster. The right cluster posseses a greater variety of topics than the left (ice hockey/association football/Austrialian football vs. baseball). So the right child cluster should be subdivided further to produce finer child clusters.

Let's give the clusters aliases as well:

In [19]:
baseball            = left_child_athletes
ice_hockey_football = right_child_athletes

### Cluster of ice hockey players and football players

In answering the following quiz question, take a look at the topics represented in the top documents (those closest to the centroid), as well as the list of words with highest TF-IDF weights.

Let us bipartition the cluster of ice hockey and football players.

In [20]:
left_child_ihs, right_child_ihs = bipartition(ice_hockey_football, maxiter=100, num_runs=6, seed=1)

In [21]:
display_single_tf_idf_cluster(left_child_ihs, map_index_to_word)

football:0.051
season:0.044
league:0.042
played:0.037
coach:0.035

* Chris Day                                          0.94236
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Ashley Prescott                                    0.94269
  ashley prescott born 11 september 1972 is a former australian rules footballer he played w
  ith the richmond and fremantle football clubs in the afl between
* Alex Lawless                                       0.94669
  alexander graham alex lawless born 26 march 1985 is a welsh professional footballer who pl
  ays for luton town as a midfielderlawless began his career with
* Lindsay Smith (Australian footballer)              0.95208
  lindsay smith born 18 july 1982 is a former australian rules footballer who played with ka
  ngaroos and carlton in the australian football leaguea tall key
* Gordon Astall                             

In [22]:
display_single_tf_idf_cluster(right_child_ihs, map_index_to_word)

tour:0.048
championships:0.047
pga:0.041
championship:0.039
won:0.034

* Tim Clark (golfer)                                 0.93468
  timothy henry clark born 17 december 1975 is a south african professional golfer who curre
  ntly plays on the pga tour his biggest win to date
* Ayelech Worku                                      0.93539
  ayelech worku born june 12 1979 is an ethiopian longdistance runner most known for winning
   two world championships bronze medals on the 5000 metres she
* Gonzalo Fern%C3%A1ndez-Casta%C3%B1o                0.94563
  gonzalo fernndezcastao born 13 october 1980 is a spanish professional golfer who plays on 
  the european tourfernndezcastao was born in madrid he turned professional in
* Richard Green (golfer)                             0.94577
  richard george green born 19 february 1971 is an australian professional golfergreen was b
  orn in williamstown melbourne victoria he turned professional in 1992 and joined
* Steven Bowditch                 

**Caution**. The granularity criteria is an imperfect heuristic and must be taken with a grain of salt. It takes a lot of manual intervention to obtain a good hierarchy of clusters.

* **If a cluster is highly mixed, the top articles and words may not convey the full picture of the cluster.** Thus, we may be misled if we judge the purity of clusters solely by their top documents and words. 
* **Many interesting topics are hidden somewhere inside the clusters but do not appear in the visualization.** We may need to subdivide further to discover new topics. For instance, subdividing the `ice_hockey_football` cluster led to the appearance of runners and golfers.

### Cluster of non-athletes

Now let us subdivide the cluster of non-athletes.

In [23]:
# Bipartition the cluster of non-athletes
left_child_non_athletes, right_child_non_athletes = bipartition(non_athletes, maxiter=100, num_runs=6, seed=1)

In [24]:
display_single_tf_idf_cluster(left_child_non_athletes, map_index_to_word)

he:0.014
music:0.013
university:0.012
film:0.011
his:0.009

* Andrew Fois                                        0.98060
  andrew fois is an attorney living and working in washington dc as of april 9 2012 he will 
  be serving as the deputy attorney general
* Third Hawkins                                      0.98077
  born maurice hawkins third hawkins is a recognized music producer in and out of the dmv ar
  ea including his hometown of baltimore maryland he has
* James A. Paul                                      0.98083
  james a paul born june 10 1941 is a writer and nonprofit executive who has worked througho
  ut his career in the field of international relations
* John Edmondson (musician)                          0.98164
  for the australian ww2 victoria cross medal award see john hurst edmondsonjohn baldwin edm
  ondson born 3 february 1933 is known throughout the world for his
* Deian Hopkin                                       0.98166
  sir deian rhys hopkin born 1 march 1

In [25]:
display_single_tf_idf_cluster(right_child_non_athletes, map_index_to_word)

she:0.141
her:0.089
miss:0.013
music:0.013
actress:0.012

* Kayee Griffin                                      0.93730
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* Ellen Christine Christiansen                       0.93783
  ellen christine christiansen born 10 december 1964 is a norwegian politician representing 
  the conservative party and formerly the progress partyborn in oslo she finished her
* Dunja Knebl                                        0.93927
  dunja knebl born 1946 is an acousticfolk singer from croatiashe was born in zagreb croatia
   but has also lived in the usa indonesia and russia
* Daphne Khoo                                        0.94136
  daphne khoo is a singaporean singer she was a contestant in the first season of singapore 
  idol finishing up in fourth place second runner up
* Priyanka Chopra                                    0.942

Neither of the clusters show clear topics, apart from the genders. Let us divide them further.

In [26]:
male_non_athletes = left_child_non_athletes
female_non_athletes = right_child_non_athletes

**Quiz Question**. Let us bipartition the clusters `male_non_athletes` and `female_non_athletes`. Which diagram best describes the resulting hierarchy of clusters for the non-athletes? Refer to the quiz for the diagrams.

**Note**. Use `maxiter=100, num_runs=6, seed=1` for consistency of output.

In [27]:
lc_male_non_athletes, rc_male_non_athletes = bipartition(male_non_athletes, maxiter=100, num_runs=6, seed=1)

In [28]:
display_single_tf_idf_cluster(lc_male_non_athletes, map_index_to_word)

music:0.026
film:0.022
band:0.016
album:0.016
art:0.015

* Third Hawkins                                      0.97234
  born maurice hawkins third hawkins is a recognized music producer in and out of the dmv ar
  ea including his hometown of baltimore maryland he has
* Anton Hecht                                        0.97393
  anton hecht is an english artist born in london in 2007 he asked musicians from around the
   durham area to contribute to a soundtrack for
* Rob Zombie                                         0.97494
  rob zombie born robert bartleh cummings january 12 1965 is an american musician film direc
  tor screenwriter and film producer he rose to prominence as a
* Stewart Levine                                     0.97586
  stewart levine is an american record producer he has worked with such artists as the crusa
  ders minnie riperton lionel richie simply red hugh masekela dr
* Tony Mills (musician)                              0.97599
  tony mills born 7 july 1962 i

In [29]:
display_single_tf_idf_cluster(rc_male_non_athletes, map_index_to_word)

university:0.017
he:0.015
research:0.015
law:0.014
president:0.014

* Andrew Fois                                        0.97357
  andrew fois is an attorney living and working in washington dc as of april 9 2012 he will 
  be serving as the deputy attorney general
* James A. Paul                                      0.97605
  james a paul born june 10 1941 is a writer and nonprofit executive who has worked througho
  ut his career in the field of international relations
* Deian Hopkin                                       0.97637
  sir deian rhys hopkin born 1 march 1944 is president of the national library of wales and 
  expert adviser to the first minister of wales
* Robert Bates (political scientist)                 0.97642
  robert hinrichs bates born 1942 is an american political scientist he is eaton professor o
  f the science of government in the departments of government and
* Parzival Copes                                     0.97658
  parzival copes oc born 22 january 1924

In [30]:
lc_female_non_athletes, rc_female_non_athletes = bipartition(female_non_athletes, maxiter=100, num_runs=6, seed=1)

In [31]:
display_single_tf_idf_cluster(lc_female_non_athletes, map_index_to_word)

she:0.168
her:0.048
election:0.036
party:0.033
council:0.028

* Kayee Griffin                                      0.89417
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* Ellen Christine Christiansen                       0.91201
  ellen christine christiansen born 10 december 1964 is a norwegian politician representing 
  the conservative party and formerly the progress partyborn in oslo she finished her
* Marcelle Mersereau                                 0.92740
  marcelle mersereau born february 14 1942 in pointeverte new brunswick is a canadian politi
  cian a civil servant for most of her career she also served
* Maureen Lyster                                     0.92745
  maureen anne lyster born 10 september 1943 is an australian politician she was an australi
  an labor party member of the victorian legislative assembly from 1985
* Ruth Ryste               

In [32]:
display_single_tf_idf_cluster(rc_female_non_athletes, map_index_to_word)

she:0.135
her:0.097
miss:0.016
music:0.016
actress:0.014

* Dunja Knebl                                        0.93789
  dunja knebl born 1946 is an acousticfolk singer from croatiashe was born in zagreb croatia
   but has also lived in the usa indonesia and russia
* Priyanka Chopra                                    0.93897
  priyanka chopra pronounced prjaka topa born 18 july 1982 is an indian film actress and sin
  ger and the winner of the miss world pageant of
* Allison Janney                                     0.94071
  allison brooks janney born november 19 1959 is an american actress she is a sixtime primet
  ime emmy award winner for her television work and has
* Daphne Khoo                                        0.94156
  daphne khoo is a singaporean singer she was a contestant in the first season of singapore 
  idol finishing up in fourth place second runner up
* Soumya Bollapragada                                0.94334
  soumya bollapragada is an indian film actresswrite